In [1]:
%%javascript
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [258]:
class AVLNode:
    
    @classmethod
    def construct_from_list(cls, avl_list, print_step = True):
        print(f'Input List:\n {avl_list}')
        avl = AVLNode(avl_list[0], print_step=print_step)
        if len(avl_list) > 1:
            for e in avl_list:  
                avl_old = avl
                avl, _ = avl.insert(e)
                avl_old.print_step = False
                avl.print_step = print_step
                #avl.print_tree()
        return avl 
    
    @classmethod
    def insert_node(cls, avl, val, print_step = True):       
        avl_old = avl            
        avl, _ = avl.insert(val)
        avl_old.print_step = False
        avl.print_step = print_step
        #return avl 
    
    def __init__(self, val, parent = None, child_type = None, print_step = False):
        self.val = val
        self.left = None
        self.right = None
        self.parent = parent
        self.child_type = child_type
        self.balance_factor = 0
        self.print_step = print_step
        
    def insert(self, val):
        
        inserted_node = None
        obj = self

        if self.val:
            if val < self.val:
                if self.left is None:
                    self.left = AVLNode(val, self, 'left')
                    inserted_node = self.left
                else:
                    _,inserted_node = self.left.insert(val)
            elif val > self.val:
                if self.right is None:
                    self.right = AVLNode(val, self, 'right')
                    inserted_node = self.right
                else:
                    _,inserted_node = self.right.insert(val)
        else:
            self.val = val
            inserted_node =self
        
        #print(inserted_node)
        bal_node = inserted_node
        while bal_node:
            bal_node.compute_balance_factor()
            bal_node = bal_node.parent
            
        if self.print_step:
            #print(inserted_node)
            print(f'\nInserting {val}\n')
            self.print_tree()
           
            
        # check  for imbalance
        imbalance_type =''
        imbalance = False
        imbalance_nodes =[]
        
        imbal_node = inserted_node
        if not self.parent:
            while imbal_node:
                if abs(imbal_node.balance_factor) >1:
                    imbalance_type = imbalance_type[-1:-3:-1]
                    print(f'\n{imbalance_type} imbalance at {imbal_node.val}')
                    imbalance_nodes.append(imbal_node)
                    imbalance_nodes = imbalance_nodes[-3:]
                    imbalance = True
                    break
                imbalance_type += imbal_node.child_type[0].upper() if imbal_node.child_type else ''
                imbalance_nodes.append(imbal_node)
                imbal_node = imbal_node.parent

            if imbalance:
                obj = AVLNode.apply_rotation(self,imbalance_type, imbalance_nodes)
                print('\nRe-evaluating balance factors\n')
                for bal_node in imbalance_nodes:
                    while bal_node:
                        bal_node.compute_balance_factor()
                        bal_node = bal_node.parent
                obj.print_tree()
            
#                 if obj != self:
#                     self.__dict__.update(obj.__dict__)
            if self.print_step:
                print('\n-------------------------------------------\n')
            return obj, None
        #print(obj)
        if self.print_step:
            print('\n-------------------------------------------\n')
        return  obj, inserted_node
    
    @classmethod
    def apply_rotation(cls , obj, imbalance_type, imbalance_nodes):
        print(f'\nImbalance type: {imbalance_type}')
        print(f'\nNodes involved: {[x.val for x in imbalance_nodes]}')
        
        if imbalance_type == 'LL':
            obj = cls.apply_R_rotation(obj, imbalance_nodes)
        elif imbalance_type == 'RR':
            obj = cls.apply_L_rotation(obj, imbalance_nodes)
        elif imbalance_type == 'LR':
            obj = cls.apply_L_rotation(obj, imbalance_nodes[:-1])
            print()
            obj.print_tree()
            print()
            imbalance_nodes = [imbalance_nodes[-1].left, imbalance_nodes[-1]]
            print([x.val for x in imbalance_nodes])
            obj = cls.apply_R_rotation(obj, imbalance_nodes)
            print()
            obj.print_tree()
            print()
        elif imbalance_type == 'RL':
            obj = cls.apply_R_rotation(obj, imbalance_nodes[:-1])   
            print()
            obj.print_tree()
            print()
            imbalance_nodes = [imbalance_nodes[-1].right, imbalance_nodes[-1]]
            print([x.val for x in imbalance_nodes])
            obj = cls.apply_L_rotation(obj, imbalance_nodes)
            print()
            obj.print_tree()
            print()
        return obj
            
    @classmethod    
    def apply_R_rotation(cls, obj, imbalance_nodes):
        print(f'\nApplying LL/ R rotation at {imbalance_nodes[-1].val}\n')
        
        imbalance_nodes[-1].left = imbalance_nodes[-2].right        
        
        imbalance_nodes[-2].right = imbalance_nodes[-1]
        imbalance_nodes[-2].parent = imbalance_nodes[-1].parent
        imbalance_nodes[-1].parent = imbalance_nodes[-2]
        imbalance_nodes[-2].child_type = imbalance_nodes[-1].child_type
        imbalance_nodes[-1].child_type = 'right'
        if imbalance_nodes[-1].left:
            imbalance_nodes[-1].left.parent = imbalance_nodes[-1]
            imbalance_nodes[-1].left.child_type = 'left'
        
        if imbalance_nodes[-2].child_type == 'left':
            imbalance_nodes[-2].parent.left = imbalance_nodes[-2]
        elif imbalance_nodes[-2].child_type == 'right':
            imbalance_nodes[-2].parent.right = imbalance_nodes[-2]
        if imbalance_nodes[-1] == obj:
            obj = imbalance_nodes[-2]
            #print('yaya')
            #self = imbalance_nodes[-2]            
            #self.__dict__.update(imbalance_nodes[-2].__dict__)
        return obj
    
    @classmethod
    def apply_L_rotation(cls, obj, imbalance_nodes):
        print(f'\nApplying RR/ L rotation at {imbalance_nodes[-1].val}\n')
        
        imbalance_nodes[-1].right = imbalance_nodes[-2].left
        imbalance_nodes[-2].left = imbalance_nodes[-1]
        imbalance_nodes[-2].parent = imbalance_nodes[-1].parent
        imbalance_nodes[-1].parent = imbalance_nodes[-2]
        imbalance_nodes[-2].child_type = imbalance_nodes[-1].child_type
        imbalance_nodes[-1].child_type = 'left'
        if imbalance_nodes[-1].right:
            imbalance_nodes[-1].right.parent = imbalance_nodes[-1]
            imbalance_nodes[-1].right.child_type = 'right'
        
        if imbalance_nodes[-2].child_type == 'left':
            imbalance_nodes[-2].parent.left = imbalance_nodes[-2]
        elif imbalance_nodes[-2].child_type == 'right':
            imbalance_nodes[-2].parent.right = imbalance_nodes[-2]
        if imbalance_nodes[-1] ==obj:
           # print('yaya')
            #self = imbalance_nodes[-2] 
            obj = imbalance_nodes[-2]
            #self.__dict__.update(imbalance_nodes[-2].__dict__)
        return obj
        
    def compute_balance_factor(self):
         
        # case - 1: node is a leaf node
        if not self.left and not self.right:
            self.balance_factor = 0
        # case - 2: node has single child
        elif not self.left and self.right: 
            self.balance_factor = -self.right.cal_height()

        elif self.left and not self.right:
            self.balance_factor = self.left.cal_height()
        # case - 3: node has both children
        elif self.left and self.right:
            self.balance_factor = self.left.cal_height() -self.right.cal_height()
            
    def cal_height(self):
        if not self.left and not self.right:
            return 1
        elif not self.left and self.right:
            return 1+ self.right.cal_height()
        elif self.left and not self.right:
            return 1+ self.left.cal_height()
        elif self.left and self.right:
            return 1+ max(self.left.cal_height(), self.right.cal_height())
        
            
            
    def find_element(self, val):
        if self.val:
            if self.val == val:               
                return self
            elif self.val < val:
                if self.right:
                    return self.right.find_element(val)
                else:
                    return None
            else:
                if self.left:
                    return self.left.find_element(val)
                else:
                    return None
        else:
            return None
    def find_maximum(self):
        x = self
        while x.right:
            x = x.right
        return x
    
    def find_minimum(self):
        x = self
        while x.left:
            x = x.left
        return x
    
    @classmethod   
    def delete(cls, obj, val):
        node = obj.find_element(val)
        node_parent = node.parent
        if obj.print_step:
            obj.print_tree()
        if node:
            # case - 1: node is a leaf node
            if not node.left and not node.right:
                if obj.print_step:
                    print(f'\nNode {val} has no children(leaf)')
                bal_node = node.parent
                node.del_node()
                
            # case - 2: node has single child
            elif not node.left and node.right: 
                if obj.print_step:
                    print(f'\nNode {val} has single child (right)')
                bal_node = node.parent    
                node.right.short_circuit()
                    
            elif node.left and not node.right:
                if obj.print_step:
                    print(f'\nNode {val} has single child (left)')
                bal_node = node.parent
                node.left.short_circuit()

                #node.del_node()
            # case - 3: node has both children
            elif node.left and node.right:
                if obj.print_step:
                    print(f'\nNode {val} has both children')
                    print(f'\nFinding successor of {val}')
                successor = node.find_successor()                
                node.val = successor.val
                if obj.print_step:
                    print(f'\nSuccessor is {successor.val}')
                    print(f'\nReplacing {val} with {successor.val}\n')
                    obj.print_tree()
                if obj.print_step:
                    print(f'\nDeleting {successor.val}')
                bal_node = successor.parent
                cls.delete(successor, successor.val)
                
            while bal_node:
                    bal_node.compute_balance_factor()
                    bal_node = bal_node.parent
        else:
            print(f'element {val} not found in the tree')
            
        if obj.print_step:
            obj.print_tree()
            
            # check  for imbalance
            imbalance_type =''
            imbalance = False
            imbalance_nodes =[]        
        
            #obj.print_tree()
            node_parent_left = node_parent
            while node_parent_left and node_parent_left.left:
                node_parent_left = node_parent_left.left            
            
            imbal_node = node_parent_left
            while imbal_node:
                if abs(imbal_node.balance_factor) >1:
                    imbalance_type = imbalance_type[-1:-3:-1]
                    print(f'\n{imbalance_type} imbalance at {imbal_node.val}')
                    imbalance_nodes.append(imbal_node)
                    imbalance_nodes = imbalance_nodes[-3:]
                    imbalance = True
                    break
                imbalance_type += imbal_node.child_type[0].upper() if imbal_node.child_type else ''
                imbalance_nodes.append(imbal_node)
                imbal_node = imbal_node.parent

            if imbalance:
                obj = AVLNode.apply_rotation(obj,imbalance_type, imbalance_nodes)
                obj.print_tree()
                print('\nRe-evaluating balance factors\n')
                for bal_node in imbalance_nodes:
                    while bal_node:
                        bal_node.compute_balance_factor()
                        bal_node = bal_node.parent
                obj.print_tree()
            
            imbalance_type =''
            imbalance = False
            imbalance_nodes =[] 
                
            node_parent_right = node_parent
            while node_parent_right and node_parent_right.right:
                node_parent_right = node_parent_right.right           
            
            imbal_node = node_parent_right
            while imbal_node:
                if abs(imbal_node.balance_factor) >1:
                    imbalance_type = imbalance_type[-1:-3:-1]
                    print(f'\n{imbalance_type} imbalance at {imbal_node.val}')
                    imbalance_nodes.append(imbal_node)
                    imbalance_nodes = imbalance_nodes[-3:]
                    imbalance = True
                    break
                imbalance_type += imbal_node.child_type[0].upper() if imbal_node.child_type else ''
                imbalance_nodes.append(imbal_node)
                imbal_node = imbal_node.parent

            if imbalance:
                obj = AVLNode.apply_rotation(obj,imbalance_type, imbalance_nodes)
                obj.print_tree()
                print('\nRe-evaluating balance factors\n')
                for bal_node in imbalance_nodes:
                    while bal_node:
                        bal_node.compute_balance_factor()
                        bal_node = bal_node.parent
                obj.print_tree()

#                 if obj != self:
#                     self.__dict__.update(obj.__dict__)
        return obj
    
    @classmethod
    def delete_node(cls, avl, val, print_step=True):
        avl_old = avl            
        avl = cls.delete(avl, val)
        avl_old.print_step = False
        avl.print_step = print_step
        
            
    def short_circuit(self):  
        if self.print_step:
             print(f'Short circuiting  {self.val}')
        if self.parent.child_type == 'right':            
            self.parent.parent.right = self
            self.child_type = 'right'
        elif self.parent.child_type == 'left':             
            self.parent.parent.left = self
            self.child_type = 'left'
        self.parent = self.parent.parent
                    
    def find_successor(self):
        if self.right:
            return self.right.find_minimum()
        else:
            return self
    def find_predessor(self):
        if self.left:
            return self.left.find_maximum()
        else:
            return self
    
    def go_left(self):
        if self.left:
            return self.left.go_left()
        else:
            return self
    
    def del_node(self):
        if self.print_step:
            print(f'\nRemoving Node {val}')
        self.val = None
        if self.child_type:
            if self.child_type == 'left':
                self.parent.left = None
            else:
                self.parent.right = None
        self.parent = None
    
    def print_tree(self):
        lines, *_ = self.display()
        for line in lines:
            print(line)
            
        
    def display(self):        
        bf_dict = {0: '\u2070', 1: '\u00B9', 2:'\u00B2', -1: '\u207B\u00B9', -2:'\u207B\u00B2'}
        
        # No child.
        if not self.left and not self.right:
            line = str(self.val) + bf_dict.get(self.balance_factor)
            width = len(line)
            height = 1
            middle = width // 2
            return [line], width, height, middle

        # Only left child.
        elif self.left and not self.right:
            lines, n, p, x = self.left.display()
            s = str(self.val) + bf_dict.get(self.balance_factor)
            u = len(s)
            first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s
            second_line = x * ' ' + '/' + (n - x - 1 + u) * ' '
            shifted_lines = [line + u * ' ' for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, n + u // 2

        # Only right child.
        elif not self.left and self.right:
            lines, n, p, x = self.right.display()
            s = str(self.val) + bf_dict.get(self.balance_factor)
            u = len(s)
            first_line = s + x * '_' + (n - x) * ' '
            second_line = (u + x) * ' ' + '\\' + (n - x - 1) * ' '
            shifted_lines = [u * ' ' + line for line in lines]
            return [first_line, second_line] + shifted_lines, n + u, p + 2, u // 2

        # Two children.
        else:
            left, n, p, x = self.left.display()
            right, m, q, y = self.right.display()
            s = str(self.val) + bf_dict.get(self.balance_factor)
            u = len(s)
            first_line = (x + 1) * ' ' + (n - x - 1) * '_' + s + y * '_' + (m - y) * ' '
            second_line = x * ' ' + '/' + (n - x - 1 + u + y) * ' ' + '\\' + (m - y - 1) * ' '
            if p < q:
                left += [n * ' '] * (q - p)
            elif q < p:
                right += [m * ' '] * (p - q)
            zipped_lines = zip(left, right)
            lines = [first_line, second_line] + [a + u * ' ' + b for a, b in zipped_lines]
            return lines, n + m + u, max(p, q) + 2, n + u // 2
           
            


In [259]:
avl = AVLNode.construct_from_list([15,3,10,2,8,13,44,9])

Input List:
 [15, 3, 10, 2, 8, 13, 44, 9]

Inserting 15

15⁰

-------------------------------------------


Inserting 3

  15¹
 /   
3⁰   

-------------------------------------------


Inserting 10

  ____15²
 /       
3⁻¹_     
    \    
   10⁰   

LR imbalance at 15

Imbalance type: LR

Nodes involved: [10, 3, 15]

Applying RR/ L rotation at 3


     _15²
    /    
  _10⁰   
 /       
3⁻¹      

[10, 15]

Applying LL/ R rotation at 15


  _10⁰_  
 /     \ 
3⁻¹   15²


Re-evaluating balance factors

  10⁰_  
 /    \ 
3⁰   15⁰

-------------------------------------------


Inserting 2

    10¹_  
   /    \ 
  3¹   15⁰
 /        
2⁰        

-------------------------------------------


Inserting 8

    __10¹_  
   /      \ 
  3⁰_    15⁰
 /   \      
2⁰  8⁰      

-------------------------------------------


Inserting 13

    __10⁰____  
   /         \ 
  3⁰_      _15¹
 /   \    /    
2⁰  8⁰   13⁰   

-------------------------------------------


Inserting 44

    __10⁰____     
   / 

In [260]:
AVLNode.insert_node(avl,4)


Inserting 4

    _______10¹____     
   /              \    
  3⁻¹___        _15⁰_  
 /      \      /     \ 
2⁰     8⁰_    13⁰   44⁰
      /   \            
     4⁰  9⁰            

-------------------------------------------



In [261]:
AVLNode.insert_node(avl,1)


Inserting 1

      ______10¹____     
     /             \    
    3⁰___        _15⁰_  
   /     \      /     \ 
  2¹    8⁰_    13⁰   44⁰
 /     /   \            
1⁰    4⁰  9⁰            

-------------------------------------------



In [262]:
AVLNode.insert_node(avl,0)


Inserting 0

        ______10²____     
       /             \    
      3¹___        _15⁰_  
     /     \      /     \ 
    2²    8⁰_    13⁰   44⁰
   /     /   \            
  1¹    4⁰  9⁰            
 /                        
0⁰                        

LL imbalance at 2

Imbalance type: LL

Nodes involved: [0, 1, 2]

Applying LL/ R rotation at 2


Re-evaluating balance factors

        ______10¹____     
       /             \    
    __3⁰___        _15⁰_  
   /       \      /     \ 
  1⁰_     8⁰_    13⁰   44⁰
 /   \   /   \            
0⁰  2⁰  4⁰  9⁰            

-------------------------------------------



In [263]:
AVLNode.delete_node(avl, 9)

        ______10¹____     
       /             \    
    __3⁰___        _15⁰_  
   /       \      /     \ 
  1⁰_     8⁰_    13⁰   44⁰
 /   \   /   \            
0⁰  2⁰  4⁰  9⁰            

Node 9 has no children(leaf)
        ____10¹____     
       /           \    
    __3⁰___      _15⁰_  
   /       \    /     \ 
  1⁰_     8¹   13⁰   44⁰
 /   \   /              
0⁰  2⁰  4⁰              


In [264]:
AVLNode.delete_node(avl, 4)

        ____10¹____     
       /           \    
    __3⁰___      _15⁰_  
   /       \    /     \ 
  1⁰_     8¹   13⁰   44⁰
 /   \   /              
0⁰  2⁰  4⁰              

Node 4 has no children(leaf)
        __10¹____     
       /         \    
    __3¹_      _15⁰_  
   /     \    /     \ 
  1⁰_   8⁰   13⁰   44⁰
 /   \                
0⁰  2⁰                


In [265]:
AVLNode.delete_node(avl, 3)

        __10¹____     
       /         \    
    __3¹_      _15⁰_  
   /     \    /     \ 
  1⁰_   8⁰   13⁰   44⁰
 /   \                
0⁰  2⁰                

Node 3 has both children

Finding successor of 3

Successor is 8

Replacing 3 with 8

        __10¹____     
       /         \    
    __8¹_      _15⁰_  
   /     \    /     \ 
  1⁰_   8⁰   13⁰   44⁰
 /   \                
0⁰  2⁰                

Deleting 8
        10¹____     
       /       \    
    __8²     _15⁰_  
   /        /     \ 
  1⁰_      13⁰   44⁰
 /   \              
0⁰  2⁰              

LL imbalance at 8

Imbalance type: LL

Nodes involved: [0, 1, 8]

Applying LL/ R rotation at 8

    ____10¹____     
   /           \    
  1⁰___      _15⁰_  
 /     \    /     \ 
0⁰    8²   13⁰   44⁰
     /              
    2⁰              

Re-evaluating balance factors

    _____10¹____     
   /            \    
  1⁻¹___      _15⁰_  
 /      \    /     \ 
0⁰     8¹   13⁰   44⁰
      /              
     2⁰              


In [266]:
avl = AVLNode.construct_from_list([64,1,14,26,13,110,98,85])

Input List:
 [64, 1, 14, 26, 13, 110, 98, 85]

Inserting 64

64⁰

-------------------------------------------


Inserting 1

  64¹
 /   
1⁰   

-------------------------------------------


Inserting 14

  ____64²
 /       
1⁻¹_     
    \    
   14⁰   

LR imbalance at 64

Imbalance type: LR

Nodes involved: [14, 1, 64]

Applying RR/ L rotation at 1


     _64²
    /    
  _14⁰   
 /       
1⁻¹      

[14, 64]

Applying LL/ R rotation at 64


  _14⁰_  
 /     \ 
1⁻¹   64²


Re-evaluating balance factors

  14⁰_  
 /    \ 
1⁰   64⁰

-------------------------------------------


Inserting 26

  14⁻¹____  
 /        \ 
1⁰      _64¹
       /    
      26⁰   

-------------------------------------------


Inserting 13

  ____14⁰____  
 /           \ 
1⁻¹_       _64¹
    \     /    
   13⁰   26⁰   

-------------------------------------------


Inserting 110

  ____14⁰____      
 /           \     
1⁻¹_       _64⁰__  
    \     /      \ 
   13⁰   26⁰   110⁰

--------------------------------

In [267]:
AVLNode.insert_node(avl,4)


Inserting 4

  ______14⁰_____            
 /              \           
1⁻²___       _64⁻¹____      
      \     /         \     
     13¹   26⁰      _98⁰__  
    /              /      \ 
   4⁰             85⁰   110⁰

RL imbalance at 1

Imbalance type: RL

Nodes involved: [4, 13, 1]

Applying LL/ R rotation at 13


  ______14⁰_____            
 /              \           
1⁻²_         _64⁻¹____      
    \       /         \     
   4⁰_     26⁰      _98⁰__  
      \            /      \ 
     13¹          85⁰   110⁰

[4, 1]

Applying RR/ L rotation at 1


     ___14⁰_____            
    /           \           
  _4⁰_       _64⁻¹____      
 /    \     /         \     
1⁻²  13¹   26⁰      _98⁰__  
                   /      \ 
                  85⁰   110⁰


Re-evaluating balance factors

    ___14⁻¹_____            
   /            \           
  4⁰_        _64⁻¹____      
 /   \      /         \     
1⁰  13⁰    26⁰      _98⁰__  
                   /      \ 
                  85⁰   110⁰



In [268]:
AVLNode.delete_node(avl, 98)

    ___14⁻¹_____            
   /            \           
  4⁰_        _64⁻¹____      
 /   \      /         \     
1⁰  13⁰    26⁰      _98⁰__  
                   /      \ 
                  85⁰   110⁰

Node 98 has both children

Finding successor of 98

Successor is 110

Replacing 98 with 110

    ___14⁻¹_____             
   /            \            
  4⁰_        _64⁻¹_____      
 /   \      /          \     
1⁰  13⁰    26⁰      _110⁰__  
                   /       \ 
                  85⁰    110⁰

Deleting 110
    ___14⁻¹_____         
   /            \        
  4⁰_        _64⁻¹_____  
 /   \      /          \ 
1⁰  13⁰    26⁰      _110¹
                   /     
                  85⁰    


In [269]:
AVLNode.delete_node(avl, 14)

    ___14⁻¹_____         
   /            \        
  4⁰_        _64⁻¹_____  
 /   \      /          \ 
1⁰  13⁰    26⁰      _110¹
                   /     
                  85⁰    

Node 14 has both children

Finding successor of 14

Successor is 26

Replacing 14 with 26

    ___26⁻¹_____         
   /            \        
  4⁰_        _64⁻¹_____  
 /   \      /          \ 
1⁰  13⁰    26⁰      _110¹
                   /     
                  85⁰    

Deleting 26
    ___26⁻¹__         
   /         \        
  4⁰_      64⁻²_____  
 /   \              \ 
1⁰  13⁰          _110¹
                /     
               85⁰    


In [270]:
avl.print_tree()

    ___26⁻¹__         
   /         \        
  4⁰_      64⁻²_____  
 /   \              \ 
1⁰  13⁰          _110¹
                /     
               85⁰    


In [271]:
AVLNode.delete_node(avl, 85)

    ___26⁻¹__         
   /         \        
  4⁰_      64⁻²_____  
 /   \              \ 
1⁰  13⁰          _110¹
                /     
               85⁰    

Node 85 has no children(leaf)
    ___26⁰__      
   /        \     
  4⁰_     64⁻¹__  
 /   \          \ 
1⁰  13⁰       110⁰
